In [1]:
import sys
import os

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from PyQt5.QtWidgets import QMainWindow, QWidget, QGridLayout, QVBoxLayout, QHBoxLayout, QApplication
from pypylon import pylon

from apps.manager import initManagers, WidgetManager, ConfigManager, DataManager, ControlManager, LayoutManager
from apps.control import ViewControl, BaslerCameraControl, ConnectorControl
from apps.layout.basler_camera_gui import makeLayoutBaslerCameraGUI
from apps.bind_func.utils import *
from apps.bind_func.basler_camera_gui import *

class BaslerCameraGUI(QMainWindow):
    def __init__(self):
        QMainWindow.__init__(self)
        self.app_name = "BaslerCameraGUI"
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager, self.layout_manager = initManagers(
            WidgetManager(), ConfigManager(), DataManager(), ControlManager(), LayoutManager()
        )

        # set window title and geometry
        self.setWindowTitle(self.app_name)
        # self.setGeometry(0, 0, 1920, 900) # hardcoded !!!

        self.initUI()

        # setup managers
        self.setupManagers()

        # setup controls
        self.setupControls()

        # bind functions
        self.bindFuncAllWidgets()

    """
    setup UI function
    """
    def initUI(self):
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QGridLayout(self.central_widget)
        self.layout_main.addLayout(
            makeLayoutBaslerCameraGUI(self.widget_manager, self.config_manager),
            0, 0
        )

    def setupManagers(self):
        # ConfigManager
        self.config_manager.current_app = self.app_name

    def setupControls(self):
        # view control
        self.control_manager.view_control = ViewControl(
            self.widget_manager.dict_view["view_camera"],
            self.widget_manager.dict_scene["view_camera"],
            self.data_manager,
            self.widget_manager,
            self.config_manager,
            self.control_manager
        )
        # add camera view layer
        self.control_manager.view_control.addViewLayer("camera")

        # setup CameraControl
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        converter = pylon.ImageFormatConverter()
        self.control_manager.camera_control = BaslerCameraControl(
            data_manager=self.data_manager,
            camera=camera,
            converter=converter,
        )
        self.control_manager.camera_control.setFlag("is_capture", False)
    
    """
    bindFunc
    """
    def bindFuncAllWidgets(self):
        # exit button
        bindFuncExit(q_button=self.widget_manager.dict_button["exit"], q_window=self)
        # single capture button
        bindFuncButtonCaptureSingle(
            self.widget_manager.dict_button["capture_single"],
            self,
            self.widget_manager,
            self.control_manager.camera_control,
            self.control_manager.view_control,
            )
        # play button
        bindFuncButtonPlay(
            self.widget_manager.dict_button["play"],
            self,
            self.widget_manager,
            self.control_manager.camera_control,
            )
        # Capture with Bpod button


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    gui = BaslerCameraGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\share\anaconda3\envs\camera\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
gui.control_manager.camera_control.dict_camera_config

{'fps': 60.0,
 'width': 1280,
 'height': 1024,
 'offset_x': 0,
 'offset_y': 0,
 'gain': 12.0,
 'exposure_time': 2000}

In [7]:
gui.widget_manager.dict_scene

{'view_camera': <PyQt5.QtWidgets.QGraphicsScene at 0x2340409d280>}